In [1]:
import sys
sys.path.append('/workspaces/AIRFLOW_MUSIC')
from util.utils import spark_session


In [ ]:
def extract_song(cur, path) :

    # spark_session = spark_session()     

    # for root, dir, file in os.walk('/workspaces/AIRFLOW_MUSIC/data/song_data'):
    #     for f in file: 

    df = spark_session.read.json(str(os.path.join(root, f)) ) 
    
    # insert song record
    df = spark_session.read.json(path)
    song_data = tuple((df.select(['song_id', 'title', 'artist_id', 'year', 'duration']).collect()[0]))
    cur.execute(song_table_insert, song_data)


    # insert artist record
    df = spark_session.read.json(path)
    song_data = tuple((df.select(['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']).collect()[0]))
    cur.execute(artist_table_insert, song_data)



In [ ]:
def process_log_file(cur, filepath):
     
    # open log file
    df = spark_session.read.json("/workspaces/AIRFLOW_MUSIC/data/log_data/2018/11/2018-11-01-events.json")

    # filter by NextSong action
    df = df.filter(df["page"] == "NextSong")

    # convert timestamp column to datetime
    t = pd.to_datetime(df['ts'])

    # insert time data records
    time_data = [(tt.value, tt.hour, tt.day, tt.week, tt.month, tt.year, tt.weekday()) for tt in t]
    column_labels = ('timestamp', 'hour', 'day', 'week', 'month', 'year', 'weekday')
    time_df = pd.DataFrame(data=time_data, columns=column_labels)

    for i, row in time_df.iterrows():
        cur.execute(time_table_insert, list(row))


In [2]:
def format_datetime(ts):

    """
    Description: converts numeric timestamp to datetime format.
    Returns:
        timestamp with type datetime
    """

    import datetime
    return datetime.datetime.fromtimestamp(ts/1000.0)

spark_session = spark_session()

df = spark_session.read.json("/workspaces/AIRFLOW_MUSIC/data/log_data/2018/11/2018-11-01-events.json")


from pyspark.sql.types import DateType , TimestampType 
from pyspark.sql.functions import udf
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, dayofweek

# create timestamp column from original timestamp column
get_timestamp = udf(lambda x: format_datetime(int(x)), TimestampType())
df = df.withColumn("start_time", get_timestamp(df.ts))

# create datetime column from original timestamp column
get_datetime = udf(lambda x: format_datetime(int(x)), DateType())
df = df.withColumn("datetime", get_datetime(df.ts))

# extract columns to create time table
time_table = df.select( 'start_time', 'datetime' , 
                        hour("datetime").alias('hour'),
                        dayofmonth("datetime").alias('day'),
                        weekofyear("datetime").alias('week'),
                        year("datetime").alias('year'),
                        month("datetime").alias('month'),
                        dayofweek("datetime").alias('weekday') )

23/01/13 19:09:11 WARN Utils: Your hostname, codespaces-a6bdc4 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
23/01/13 19:09:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/13 19:09:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
